In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
!nvcc --version

In [0]:
!pip install tensorflow-gpu==1.12.0
import tensorflow as tf
tf.test.gpu_device_name()

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### tar.gz file with clean, distorted and test_distorted directories should be available under the path in the Google Drive

In [0]:
!tar -xvf 'gdrive/My Drive/UJ/AppliedDL/image_denoiser_project3/data.tar.gz'

In [7]:
!ls
!mkdir data
!mv clean ./data
!mv distorted ./data
!mv test_distorted ./data
!ls data

clean						    distorted	 test_distorted
cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb  gdrive	 ujnn2019-3.zip
data						    sample_data
mkdir: cannot create directory ‘data’: File exists
mv: cannot move 'clean' to './data/clean': Directory not empty
mv: cannot move 'distorted' to './data/distorted': Directory not empty
mv: cannot move 'test_distorted' to './data/test_distorted': Directory not empty
clean  distorted  test_distorted


In [8]:
from keras.preprocessing.image import load_img, array_to_img, img_to_array, ImageDataGenerator
import numpy as np
import os
import warnings
def ignore_warn(*args, **kwargs):
    pass
  
!ls data

Using TensorFlow backend.


clean  distorted  test_distorted


In [0]:
root_path = './data'

train_path = root_path + '/distorted'
train_cleaned_path = root_path + '/clean'
test_path = root_path + '/test_distorted'

train_images = sorted(os.listdir(train_path))
train_labels = sorted(os.listdir(train_cleaned_path))
test_images = sorted(os.listdir(test_path))

In [10]:
X = []
Y = []
X_test=[]
print("Load train images...")
for i, img in enumerate(train_images):
    img = load_img(train_path + '/' + img,target_size=(48,48,3))
    img = img_to_array(img).astype('float32')/255.
    X.append(img)
    if i % 2000 == 0:
      print(i)

print("Load label images...")
for i, img in enumerate(train_labels):
    img = load_img(train_cleaned_path + '/' + img,target_size=(48,48,3))
    img = img_to_array(img).astype('float32')/255.
    Y.append(img)
    if i % 2000 == 0:
      print(i)

print("Load test images...")
for i, img in enumerate(test_images):
    img = load_img(test_path + '/' + img,target_size=(48,48,3))
    img = img_to_array(img).astype('float32')/255.
    X_test.append(img)
    if i % 2000 == 0:
      print(i)

X_test = np.array(X_test)
X = np.array(X)
Y = np.array(Y)

X.shape,Y.shape, X_test.shape

Load train images...
0
2000
4000
6000
8000
Load label images...
0
2000
4000
6000
8000
Load test images...
0


((10000, 48, 48, 3), (10000, 48, 48, 3), (400, 48, 48, 3))

In [0]:
data_gen_args = dict(
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

image_datagen = ImageDataGenerator(**data_gen_args)
image_clean_datagen = ImageDataGenerator(**data_gen_args)

In [0]:
seed = 12
BATCH_SIZE=64

image_datagen.fit(X, augment=True, seed=seed)
image_clean_datagen.fit(Y, augment=True, seed=seed)

image_generator = image_datagen.flow(X, batch_size=BATCH_SIZE, seed=seed)
image_clean_generator = image_clean_datagen.flow(Y, batch_size=BATCH_SIZE, seed=seed)

In [0]:
# !mkdir preview
# !mkdir preview/training
# !mkdir preview/labels

# seed = 1
# img = X[0]
# img = img.reshape((1,) + img.shape)
# img2 = Y[0]
# img2 = img2.reshape((1,) + img2.shape)
# image_datagen.fit(X, augment=True, seed=seed)
# image_clean_datagen.fit(Y, augment=True, seed=seed)
# i = 0
# for batch in image_datagen.flow(img, batch_size=1,
#                           save_to_dir='preview/training', save_prefix='train', save_format='jpeg', seed=1):
#     i += 1
#     if i > 5:
#         break  # otherwise the generator would loop indefinitely   
        
# i = 0
# for batch in image_clean_datagen.flow(img2, batch_size=1,
#                           save_to_dir='preview/labels', save_prefix='label', save_format='jpeg', seed=1):
#     i += 1
#     if i > 5:
#         break  # otherwise the generator would loop indefinitely

In [0]:
!ls

In [0]:
!rm -r preview

In [0]:
!cp -r preview 'gdrive/My Drive/UJ/AppliedDL/image_denoiser_project3/'

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, UpSampling2D, Conv2DTranspose, BatchNormalization
from tensorflow.keras import Model
# Model
def make_convolutional_autoencoder():
    # encoding
    inputs = Input(shape=(48, 48, 3))
    x = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(padding='same')(x)
    
    x = Conv2D(256, 3, activation='relu', padding='same')(x)
    encoded = BatchNormalization()(x)
    
    
    # decoding
    x = Conv2DTranspose(256, 3, strides=(1, 1), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(128, 3, strides=(1, 1), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(128, 3, strides=(1, 1), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(64, 3, strides=(1, 1), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(64, 3, strides=(2, 2), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(32, 3, strides=(1, 1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(16, 3, strides=(2, 2), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    decoded = Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)
    
    # autoencoder
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', 
                        loss='mean_squared_error')
    return autoencoder
  
# create a convolutional autoencoder
autoencoder = make_convolutional_autoencoder()

In [0]:
train_generator = zip(image_generator, image_clean_generator)

autoencoder.fit_generator(
    train_generator,
    steps_per_epoch=2000,
    epochs=20)

Epoch 1/20
2000/2000 [==============================] - 195s 97ms/step - loss: 0.2767
Epoch 2/20
2000/2000 [==============================] - 191s 95ms/step - loss: 0.1633
Epoch 3/20
2000/2000 [==============================] - 190s 95ms/step - loss: 0.1495
Epoch 4/20
2000/2000 [==============================] - 191s 96ms/step - loss: 0.1420
Epoch 5/20
2000/2000 [==============================] - 192s 96ms/step - loss: 0.1373
Epoch 6/20
2000/2000 [==============================] - 191s 96ms/step - loss: 0.1339
Epoch 7/20
1610/2000 [=======================>......] - ETA: 37s - loss: 0.1314

In [0]:
!rm -r result

In [60]:
!mkdir result
!mkdir result/base
!mkdir result/example

base = X[16]
base = base.reshape((1,) + base.shape)
example = autoencoder.predict(base)

base_image_generator = ImageDataGenerator()
base_image_generator.fit(base)

example_image_generator = ImageDataGenerator()
example_image_generator.fit(example)

i = 0
for batch in image_datagen.flow(base, batch_size=1,
                          save_to_dir='result/base', save_prefix='base', save_format='jpeg', seed=1):
    i += 1
    if i > 0:
        break  # otherwise the generator would loop indefinitely

i = 0     
for batch in image_datagen.flow(example, batch_size=1,
                          save_to_dir='result/example', save_prefix='example', save_format='jpeg', seed=1):
    i += 1
    if i > 0:
        break  # otherwise the generator would loop indefinitely


mkdir: cannot create directory ‘result’: File exists
mkdir: cannot create directory ‘result/base’: File exists
mkdir: cannot create directory ‘result/example’: File exists


In [61]:
!ls result/example
!cp -r result 'gdrive/My Drive/UJ/AppliedDL/image_denoiser_project3/'

example_0_3462.jpeg


In [0]:
tf.keras.models.save_model(
    model=autoencoder,
    filepath='gdrive/My Drive/UJ/AppliedDL/image_denoiser_project3/model_2000.h5'
)

In [0]:
model = tf.keras.models.load_model('gdrive/My Drive/UJ/AppliedDL/image_denoiser_project3/model_2000.h5')


In [80]:
!mkdir result
!mkdir result/base
!mkdir result/example

base = X[11]
base = base.reshape((1,) + base.shape)
example = model.predict(base)

base_image_generator = ImageDataGenerator()
base_image_generator.fit(base)

example_image_generator = ImageDataGenerator()
example_image_generator.fit(example)

i = 0
for batch in image_datagen.flow(base, batch_size=1,
                          save_to_dir='result/base', save_prefix='base', save_format='jpeg', seed=1):
    i += 1
    if i > 0:
        break  # otherwise the generator would loop indefinitely

i = 0     
for batch in image_datagen.flow(example, batch_size=1,
                          save_to_dir='result/example', save_prefix='example', save_format='jpeg', seed=1):
    i += 1
    if i > 0:
        break  # otherwise the generator would loop indefinitely

mkdir: cannot create directory ‘result’: File exists
mkdir: cannot create directory ‘result/base’: File exists
mkdir: cannot create directory ‘result/example’: File exists


In [81]:
!ls result/example
!cp -r result 'gdrive/My Drive/UJ/AppliedDL/image_denoiser_project3/'

example_0_3462.jpeg


In [88]:
X.shape

(10000, 48, 48, 3)